In [2]:
# -*- coding: utf8 -*-

In [3]:
debug=False

filePrefix="MGC-160104"
nomLexique="/Users/gilles/Copy/Python/phonemisation/bdlexique-PDM-Resync2.txt"
nomConverts="/Users/gilles/Github/SWIM/2015-Data/VerbesAction-ConvertsPP.txt"
nomAutres="/Users/gilles/Github/SWIM/2015-Data/VerbesAction-Autres2.txt"

freqForme=1000000

## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [4]:
import pandas as pd
import pickle
import numpy as np

# Préparation du tableau de VERBES

### Lecture du lexique
- nomLexique pour le fichier
- names pour les noms de colonnes
- élimination des lignes dupliquées éventuelles (p.e. dépendre)

In [5]:
bdlexique=pd.read_csv(nomLexique,sep=";",names=["ortho","phono","ext","cs","ms","vs","lexeme","L23","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"],encoding="utf8")
bdlexique.loc[(bdlexique["ms"]=="ij")&(bdlexique["cs"].isin(["V","K"])),"ms"]="MS"
bdlexique.drop_duplicates(inplace=True,subset=["phono","ext","cs","ms","vs","lexeme","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"])

### Création de la colonne de fréquence *freq*

- fréquence forme : 1000000x(formeFrantext+formeFilms)
- fréquence lexeme : lemmeFrantext+lemmeFilms
- fréquence inexistante : .01

In [6]:
bdlexique["freq"]=1.0/100
filtreLFrantext=(bdlexique["lemmeFrantext"]!="***") 
filtreLFilms=(bdlexique["lemmeFilms"]!="***")
filtreFFrantext=(bdlexique["formeFrantext"]!="***") 
filtreFFilms=(bdlexique["formeFilms"]!="***")
bdlexique.loc[filtreLFrantext|filtreLFilms,"freq"]=bdlexique[filtreLFrantext]["lemmeFrantext"].astype(float)+bdlexique[filtreLFilms]["lemmeFilms"].astype(float)
bdlexique.loc[filtreFFrantext | filtreFFrantext,"freq"]=bdlexique[filtreFFrantext]["formeFrantext"].astype(float)*freqForme+bdlexique[filtreFFilms]["formeFilms"].astype(float)*freqForme

### Elimination des colonnes inutiles
- indicateur L23
- fréquences brutes

In [7]:
inutilesColonnes=["L23","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"]
for colonne in inutilesColonnes:
    del bdlexique[colonne]

In [8]:
convertsPP=pd.read_csv(nomConverts,sep=";",names=["lexeme","base"],encoding="utf8")
autresDerives=pd.read_csv(nomAutres,sep=";",names=["lexeme","base"],encoding="utf8")


In [9]:
derivesLexicaux=pd.concat([convertsPP,autresDerives]).reset_index()
del derivesLexicaux["index"]
derivesLexicaux=derivesLexicaux.drop_duplicates()
listDerives=derivesLexicaux["lexeme"].tolist()

### Extraction des converts dans une structure *derives*

In [10]:
derives=bdlexique[(bdlexique["cs"]=="N") & (bdlexique["ms"].str.startswith("F")) & (bdlexique["lexeme"].isin(listDerives))]

In [11]:
derives=derives.merge(derivesLexicaux,on="lexeme",how="left")
#derives=derives.merge(autresDerives,on="lexeme",how="left")

In [12]:
derives

,ortho,phono,ext,cs,ms,vs,lexeme,freq,base
0,absoute,absut,@,N,FS,NaN,absoute,1.400000e+07,absoudre
1,absoutes,absut,"@z""",N,FP,NaN,absoute,1.400000e+01,absoudre
2,accalmie,akalmi,NaN,N,FS,NaN,accalmie,3.430000e+08,accalmir
3,accalmies,akalmi,"z""",N,FP,NaN,accalmie,7.800000e+07,accalmir
4,accrue,akry,NaN,N,FS,NaN,accrue,7.000000e+06,accroître
5,accrues,akry,"z""",N,FP,NaN,accrue,7.000000e+00,accroître
6,agonie,agòni,NaN,N,FS,NaN,agonie,1.782000e+09,agonir
7,agonies,agòni,"z""",N,FP,NaN,agonie,1.470000e+08,agonir
8,allée,ale,NaN,N,FS,NaN,allée,4.597000e+09,aller
9,allées,ale,"z""",N,FP,NaN,allée,2.855000e+09,aller


### Ajout d'une colonne *prob* pour la fréquence relative

In [13]:
derives["prob"]=derives["freq"]/derives["freq"].sum()

### Ajout d'une colonne *case*
- remplir les *ms* vides pour permettre la concaténation sans erreur
- remplir les *vs* vides des participes passés par "pp" pour permettre l'identification de la case
- *case* est la concaténation de *vs* et *ms*

In [14]:
bdlexiqueIn = unicode(u"èò")
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO")
neutralise = dict(zip(bdlexiqueNum, neutreOut))

def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

derives["phono"]=derives["phono"].apply(recoder)

In [15]:
with open("2015-Data/"+filePrefix+'-Derives2.pkl', 'wb') as output:
   pickle.dump(derives, output, pickle.HIGHEST_PROTOCOL)

In [16]:
derives.sort_values(by="prob")

,ortho,phono,ext,cs,ms,vs,lexeme,freq,base,prob
159,pavées,pave,"z""",N,FP,NaN,pavée,1.000000e-02,paver,1.281217e-14
78,déprise,dEpriz,@,N,FS,NaN,déprise,1.000000e-02,déprendre,1.281217e-14
79,déprises,dEpriz,"@z""",N,FP,NaN,déprise,1.000000e-02,déprendre,1.281217e-14
156,passée,pase,NaN,N,FS,NaN,passée,1.000000e-02,passer,1.281217e-14
157,passées,pase,"z""",N,FP,NaN,passée,1.000000e-02,passer,1.281217e-14
158,pavée,pave,NaN,N,FS,NaN,pavée,1.000000e-02,paver,1.281217e-14
196,reconduite,r6kôdHit,@,N,FS,NaN,reconduite,1.000000e-02,reconduire,1.281217e-14
197,reconduites,r6kôdHit,"@z""",N,FP,NaN,reconduite,1.000000e-02,reconduire,1.281217e-14
214,reposée,r6pOze,NaN,N,FS,NaN,reposée,1.000000e-02,reposer,1.281217e-14
215,reposées,r6pOze,"z""",N,FP,NaN,reposée,1.000000e-02,reposer,1.281217e-14


In [16]:
tirage=np.random.choice(derives.index,size=100,p=derives["prob"],replace=False)

In [17]:
extrait=derives.ix[np.sort(tirage)]

In [18]:
paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['ms'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
paradigmes

ms,lexeme,FP,FS
0,accalmie,akalmi,NaN
1,agonie,NaN,agOni
2,allée,ale,ale
3,arrivée,NaN,arive
4,atteinte,atêt,NaN
5,attente,NaN,atât
6,battue,baty,NaN
7,chute,Syt,Syt
8,conduite,NaN,kôdHit
9,conquête,kôkEt,kôkEt


In [19]:
paradigmes.to_csv(path_or_buf="2015-Data/"+filePrefix+"-derives2.csv",encoding="utf8",sep=";")

In [17]:
GOLD=pd.pivot_table(derives, values='phono', index=['lexeme'], columns=['ms'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
GOLD.to_csv(path_or_buf="2015-Data/"+filePrefix+"-derives-GOLD2.csv",encoding="utf8",sep=";")

In [18]:
GOLD

ms,lexeme,FP,FS
0,absoute,absut,absut
1,accalmie,akalmi,akalmi
2,accrue,akry,akry
3,agonie,agOni,agOni
4,allée,ale,ale
5,amenée,am6ne,am6ne
6,arrivée,arive,arive
7,assise,asiz,asiz
8,astreinte,astrêt,astrêt
9,atteinte,atêt,atêt
